In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Data/CTG.xls', sheet_name = 'Raw Data')

In [3]:
df

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Variab10.txt,1996-12-01,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
2,Fmcs_1.txt,1996-05-03,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,1996-05-03,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,1996-05-03,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,S8001045.dsp,1998-06-06,CTG2127.txt,1576.0,3049.0,140.0,140.0,1.0,0.0,9.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2126,S8001045.dsp,1998-06-06,CTG2128.txt,2796.0,3415.0,142.0,142.0,1.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2127,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2128,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.drop(index = 0, axis = 0, inplace = True)
df.drop(df.tail(3).index, inplace = True)

In [5]:
df.columns

Index(['FileName', 'Date', 'SegFile', 'b', 'e', 'LBE', 'LB', 'AC', 'FM', 'UC',
       'ASTV', 'MSTV', 'ALTV', 'MLTV', 'DL', 'DS', 'DP', 'DR', 'Width', 'Min',
       'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance',
       'Tendency', 'A', 'B', 'C', 'D', 'E', 'AD', 'DE', 'LD', 'FS', 'SUSP',
       'CLASS', 'NSP'],
      dtype='object')

In [6]:
df.drop(['FileName', 'Date', 'SegFile', 'b', 'e', 'LB','DR','Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'AC', 'FM', 'UC', 'MSTV', 'MLTV', 'Width','A', 'B', 'C', 'D', 'E', 'AD', 'DE', 'LD', 'FS', 'SUSP', 'CLASS'],axis =1, inplace = True)

In [7]:
df.columns

Index(['LBE', 'ASTV', 'ALTV', 'DL', 'DS', 'DP', 'Tendency', 'NSP'], dtype='object')

In [8]:
df['ASTV'] = df['ASTV']*10
df['ALTV'] = df['ALTV']*10
df['Tendency'] += 1

In [9]:
import skfuzzy as fuzz

In [10]:
def createLinguisticVariables(df, column):
    columnMin = df[column].min()
    columnMax = df[column].max()
    rangeDistance = columnMax - columnMin
    columnRange = np.array(range(0,(int)(columnMax + 1)))
    columnLow = fuzz.membership.zmf(columnRange,columnMin + rangeDistance*0.1,columnMin + rangeDistance*0.5)
    columnMid = fuzz.membership.trimf(columnRange, [columnMin + rangeDistance*0.25, columnMin + rangeDistance*0.5, columnMin + rangeDistance*0.75])
    columnHigh = fuzz.membership.trapmf(columnRange, [columnMin + rangeDistance*0.66,columnMin + rangeDistance*0.9, columnMax+1, columnMax+2])
    return [columnLow, columnMid, columnHigh]

In [11]:
def createMembershipDictionary(df):
    dictionary = {}
    for column in df.columns[:-1]:
        dictionary[column] = createLinguisticVariables(df,column)
    return dictionary

In [12]:
def ruleFiringStrength(rule, membershipDict, x):
    ruleLength = len(rule)
    columns = df.columns
    currentMin = 1
    isValid = False
    for index in range(0,len(rule) - 1):
        if rule[index] == -1:
            continue
        isValid = True
        currentMembership = membershipDict[columns[index]][rule[index]][int(x[columns[index]])]
        if currentMin > currentMembership:
            currentMin = currentMembership
    if isValid == False:
        return (0,rule[len(rule) - 1])
    return (currentMin, rule[len(rule) - 1])

In [13]:
def fuzzyClassifyFromRules(rules, classesNumber, membershipDict, x):
    result = [0 for i in range(classesNumber)]
    for rule in rules:
        membership, classOutput = ruleFiringStrength(rule, membershipDict, x)
        if result[classOutput] < membership:
            result[classOutput] = membership
    return result

In [14]:
def defuzzyfication(membership):
    maxValue = np.array(membership).max()
    if maxValue == 0:
        return [0]
    result = []
    for index in range(0,len(membership)):
        if membership[index] == maxValue:
            result.append(index)
    return result

In [15]:
def predictionScore(correctClass, membership):
    return correctClass in membership

In [16]:
import itertools

In [17]:
def generateAllRules(linguisticVariables, predictingClass, features):
    return [list(x) + [predictingClass] for x in list(itertools.product(range(-1,linguisticVariables), repeat=features))]

In [18]:
def fitnessFunction(rule, X_train, y_train, membershipDict):
    error = 0
    predictingClass = rule[len(rule)-1]
    for i in range(len(X_train)):
        membership, _ = ruleFiringStrength(rule, membershipDict, X_train.iloc[i])
        if y_train.iloc[i] == predictingClass + 1:
            error = error + 1 - membership
        else:
            error = error + 1 + membership
    return error

In [19]:
def calculateRulesAccuracy(rules, X_train, y_train, membershipDict):
    return [(rule,fitnessFunction(rule,X_train, y_train, membershipDict)) for rule in rules]

In [20]:
dict = createMembershipDictionary(df)

In [21]:
def modelAccuracy(model, numberClasses, membershipDict, X_test, y_test):
    correctPredicted = 0
    for i in range(len(X_test)):
        fuzzyResult = fuzzyClassifyFromRules(model, numberClasses, membershipDict,X_test.iloc[i])
        classOutputList = defuzzyfication(fuzzyResult)
        correctPredicted += predictionScore(y_test.iloc[i] - 1, classOutputList)
    return correctPredicted

In [22]:
def classAccuracy(model, predictClass, membershipDict, X_test, y_test):
    correctPredicted = 0
    allClass = 0
    for i in range(len(X_test)):
        if y_test.iloc[i] != predictClass:
            continue
        allClass += 1
        fuzzyResult = fuzzyClassifyFromRules(model,3,membershipDict,X_test.iloc[i])
        classOutputList = defuzzyfication(fuzzyResult)
        correctPredicted += predictionScore(y_test.iloc[i] - 1,classOutputList)
    return correctPredicted

In [46]:
def findBestModelForClass(rulesList, predictedClass, X_train, y_train, membershipDict):
    result = []
    model = []
    currentMax = -1
    for i in range(len(rulesList)):
        if (i%100 == 0):
            print(i)
        model.append(rulesList[i][0])
        newMax = classAccuracy(model, predictedClass, membershipDict, X_train, y_train)
        if (currentMax < newMax):
            currentMax = newMax
            result.append(i)
        else:
            model = model[:-1]
    return result

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['NSP'],axis = 1), df['NSP'], test_size=0.20, random_state=7777)

In [26]:
rulesForClass0 = calculateRulesAccuracy(generateAllRules(3,0,len(X_train.columns)), X_train, y_train, dict)

In [27]:
rulesForClass1 = calculateRulesAccuracy(generateAllRules(3,1,len(X_train.columns)),X_train, y_train, dict)

In [28]:
rulesForClass2 = calculateRulesAccuracy(generateAllRules(3,2,len(X_train.columns)),X_train, y_train, dict)

In [29]:
rulesForClass0.sort(key=lambda x:x[1])

In [30]:
rulesForClass1.sort(key=lambda x:x[1])

In [31]:
rulesForClass2.sort(key=lambda x:x[1])

In [34]:
bestRulesForClass0 = findBestModelForClass(rulesForClass0, 1, X_train, y_train, dict)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300


In [48]:
bestRulesForClass1 = findBestModelForClass(rulesForClass1, 2, X_train, y_train, dict)

0
0
16
40
64
72
100
176
186
200
234
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12545
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14425
14500
14600
14654
14700
14781
14800
14811
14900
14904
15000
15050
15100
15171
15200
15300
15370
15400
15414
15482
15500
15600
15700
15792
15800
15900
15905
16000
16022
16031
16033
16068
16087
16100
16200


In [52]:
bestRulesForClass2 = findBestModelForClass(rulesForClass2, 3, X_train, y_train, dict)

0
0
8
100
104
144
170
200
300
388
400
408
500
600
700
750
768
800
900
901
1000
1100
1200
1300
1362
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14738
14800
14861
14900
14913
15000
15001
15011
15100
15200
15232
15247
15300
15347
15400
15500
15526
15555
15600
15700
15800
15826
15900
16000
16025
16065
16069
16100
16200
16300


In [60]:
model = [rulesForClass0[index][0] for index in bestRulesForClass0] + [rulesForClass1[index][0] for index in bestRulesForClass1] + [rulesForClass2[index][0] for index in bestRulesForClass2]

In [61]:
model

[[-1, -1, 0, -1, 0, 0, -1, 0],
 [-1, 2, 1, -1, -1, -1, -1, 1],
 [-1, -1, 1, 0, -1, -1, 2, 1],
 [2, 2, -1, -1, -1, -1, 2, 1],
 [-1, -1, 1, 0, -1, -1, -1, 1],
 [2, 2, -1, 0, -1, -1, -1, 1],
 [-1, 2, -1, 0, -1, 0, 2, 1],
 [-1, -1, -1, -1, -1, 2, 2, 1],
 [-1, 0, -1, 1, -1, -1, 0, 1],
 [2, 0, -1, 1, -1, -1, 1, 1],
 [0, 2, 0, 0, 0, 0, 0, 1],
 [2, -1, -1, -1, -1, -1, 0, 1],
 [0, 2, 0, 0, -1, 0, 1, 1],
 [1, 1, -1, 0, -1, 1, 1, 1],
 [1, 2, 0, -1, 0, 0, 0, 1],
 [0, 0, -1, 1, -1, -1, 1, 1],
 [2, -1, -1, 0, -1, -1, 2, 1],
 [2, 0, 0, -1, -1, -1, 1, 1],
 [1, 2, 0, 0, -1, 0, 1, 1],
 [2, 1, 0, 0, -1, -1, 1, 1],
 [0, 1, 0, 0, 0, -1, 2, 1],
 [0, 1, 0, 0, -1, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 2, 1],
 [0, 0, 0, 0, -1, 0, 1, 1],
 [1, 1, 0, 0, 0, 0, 2, 1],
 [1, 1, 0, 0, -1, 0, 1, 1],
 [1, 0, 0, 0, -1, 0, 1, 1],
 [-1, -1, 0, -1, -1, 1, -1, 2],
 [-1, -1, 2, -1, -1, -1, -1, 2],
 [0, -1, -1, 1, -1, -1, 0, 2],
 [-1, -1, -1, -1, -1, 2, -1, 2],
 [-1, -1, -1, -1, 2, -1, -1, 2],
 [-1, 2, -1, 1, -1, -1, -1, 2],
 [0, 2, 0

In [62]:
import pickle

In [66]:
with open('rules7variables.txt','wb') as f:
    pickle.dump(model, f)

In [67]:
with open('rules7variables.txt','rb') as f:
    model2 = pickle.load(f)

In [69]:
modelAccuracy(model, 3, dict, X_train, y_train) / len(X_train)

0.8552941176470589

In [70]:
modelAccuracy(model, 3, dict, X_test, y_test) / len(X_test)

0.8591549295774648

In [77]:
def cleanRules(model, X_train, y_train, membershipDict):
    result = []
    startAccuracy = modelAccuracy(model, 3, membershipDict, X_train, y_train)
    for i in range(0,len(model)):
        newAccuracy = modelAccuracy(model[:i] + model[i+1:], 3, membershipDict, X_train, y_train)
        if (newAccuracy >= startAccuracy):
            result.append(i)
            model = model[:i] + model[i+1:]
    return model

In [78]:
cleanedModel = cleanRules(model, X_train, y_train, dict)

In [79]:
len(model)

52

In [80]:
len(cleanedModel)

30

In [81]:
modelAccuracy(cleanedModel, 3, dict, X_train, y_train) / len(X_train)

0.8552941176470589

In [82]:
modelAccuracy(cleanedModel, 3, dict, X_test, y_test) / len(X_test)

0.8591549295774648

In [85]:
cleanedModel

[[-1, -1, 0, -1, 0, 0, -1, 0],
 [-1, -1, 1, 0, -1, -1, 2, 1],
 [-1, -1, 1, 0, -1, -1, -1, 1],
 [-1, 2, -1, 0, -1, 0, 2, 1],
 [-1, -1, -1, -1, -1, 2, 2, 1],
 [2, 0, -1, 1, -1, -1, 1, 1],
 [2, -1, -1, -1, -1, -1, 0, 1],
 [1, 1, -1, 0, -1, 1, 1, 1],
 [0, 0, -1, 1, -1, -1, 1, 1],
 [2, -1, -1, 0, -1, -1, 2, 1],
 [1, 2, 0, 0, -1, 0, 1, 1],
 [0, 1, 0, 0, 0, -1, 2, 1],
 [1, 0, 0, 0, 0, 0, 2, 1],
 [1, 1, 0, 0, 0, 0, 2, 1],
 [1, 0, 0, 0, -1, 0, 1, 1],
 [-1, -1, 0, -1, -1, 1, -1, 2],
 [-1, -1, 2, -1, -1, -1, -1, 2],
 [0, -1, -1, 1, -1, -1, 0, 2],
 [-1, -1, -1, -1, -1, 2, -1, 2],
 [-1, 2, -1, 1, -1, -1, -1, 2],
 [0, -1, -1, 2, -1, -1, -1, 2],
 [-1, 2, 0, -1, -1, -1, 0, 2],
 [1, 1, 0, 0, -1, -1, 0, 2],
 [2, 2, -1, 0, -1, -1, 1, 2],
 [0, 1, -1, 1, -1, -1, 1, 2],
 [1, 2, 0, 0, -1, -1, 2, 2],
 [1, 2, 1, -1, -1, -1, 1, 2],
 [0, 0, -1, 0, -1, 0, 2, 2],
 [0, 0, 0, 0, -1, 0, 1, 2],
 [1, 0, 0, 0, -1, -1, 1, 2]]

In [93]:
def predictionScore(correctClass, membership):
    return correctClass == max(membership)

In [94]:
modelAccuracy(cleanedModel, 3, dict, X_train, y_train) / len(X_train)

0.8352941176470589

In [95]:
modelAccuracy(cleanedModel, 3, dict, X_test, y_test) / len(X_test)

0.8427230046948356

In [96]:
def ruleFiringStrength(rule, membershipDict, x):
    ruleLength = len(rule)
    columns = df.columns
    currentMin = 1
    isValid = False
    for index in range(0,len(rule) - 1):
        if rule[index] == -1:
            continue
        isValid = True
        currentMembership = membershipDict[columns[index]][rule[index]][int(x[columns[index]])]
        currentMin *= currentMembership
    if isValid == False:
        return (0,rule[len(rule) - 1])
    return (currentMin, rule[len(rule) - 1])

In [97]:
modelAccuracy(cleanedModel, 3, dict, X_train, y_train) / len(X_train)

0.8464705882352941

In [98]:
modelAccuracy(cleanedModel, 3, dict, X_test, y_test) / len(X_test)

0.8544600938967136

In [99]:
def predictionScore(correctClass, membership):
    return correctClass in membership

In [100]:
modelAccuracy(cleanedModel, 3, dict, X_train, y_train) / len(X_train)

0.8529411764705882

In [101]:
modelAccuracy(cleanedModel, 3, dict, X_test, y_test) / len(X_test)

0.8568075117370892

In [102]:
X_train.columns

Index(['LBE', 'ASTV', 'ALTV', 'DL', 'DS', 'DP', 'Tendency'], dtype='object')

In [106]:
def modelAccuracyPerClasses(model, numberClasses, membershipDict, X_test, y_test):
    dictionary = {}
    dictionary[1] = 0
    dictionary[2] = 0
    dictionary[3] = 0
    for i in range(len(X_test)):
        fuzzyResult = fuzzyClassifyFromRules(model, numberClasses, membershipDict,X_test.iloc[i])
        classOutputList = defuzzyfication(fuzzyResult)
        if predictionScore(y_test.iloc[i] - 1, classOutputList) == 0:
            dictionary[y_test.iloc[i]] += 1
    return dictionary

In [107]:
wrongDictionary = modelAccuracyPerClasses(cleanedModel, 3, dict, X_test, y_test)

In [109]:
wrongDictionary

{1: 16, 2: 37, 3: 8}